In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier,BaggingRegressor,BaggingClassifier, \
AdaBoostRegressor,AdaBoostClassifier,RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from pyearth import Earth
import itertools as it
import time as time
import xgboost as xgb
import re 

In [3]:
X_train = pd.read_csv('X_train_stratified.csv')
y_train = pd.read_csv('y_train_stratified.csv')
X_test = pd.read_csv('X_test_stratified.csv')
y_test = pd.read_csv('y_test_stratified.csv')

In [ ]:
# all models
dt_model = DecisionTreeClassifier(random_state=1, max_depth=17, max_features = 4, max_leaf_nodes=613).fit(X_train,y_train)
bg_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=1), n_estimators=300, random_state=1,max_features=7, 
                             max_samples=2500, bootstrap=False,bootstrap_features=False).fit(X_train, y_train)
rf_model = 
xgb_model = 


In [ ]:
# Train data:
dt_pred = xgb_model.predict(X_train)
bg_pred = ada_model.predict(X_train)
rf_pred = rf_model.predict(X_train)
xgb_pred = xgb_model.predict(X_train)

y_pred = (0.1*dt_pred+0.3*bg_pred+0.3*rf_pred+0.3*xgb_pred)
print(classification_report(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))

In [ ]:
# Test data:
dt_pred = xgb_model.predict(X_test)
bg_pred = ada_model.predict(X_test)
rf_pred = rf_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

y_pred = (0.1*dt_pred+0.3*bg_pred+0.3*rf_pred+0.3*xgb_pred)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))